# File Summarizer with Image Captioning

This Python class, `Summarizer`, integrates with Google Drive to download files and generate summaries or captions based on the file type. It supports handling PDF, DOCX, and image files. Here's a breakdown of what the code does:

1. **Google Drive File Download**:
   - The `download_file` method retrieves a file from Google Drive using its file ID, downloads it to the local system, and identifies its type (image, DOCX, PDF) for further processing.

2. **Image Captioning**:
   - If the file is an image (JPG, PNG, etc.), the code uses a pretrained model from the Hugging Face `transformers` library (BLIP model) to generate an image caption.

3. **PDF Summarization**:
   - For PDF files, the code extracts text using the `PyPDF2` library, and if the extracted text is present, it generates a summary using a BART-based summarization model (`facebook/bart-large-cnn`).

4. **DOCX Summarization**:
   - For DOCX files, it extracts text using the `docx` library, and then summarizes the content similarly using the BART model.

5. **Error Handling**:
   - The code handles various errors related to file download, text extraction, and summarization. It also prints progress updates for file downloads.

Overall, the code facilitates downloading, summarizing, and captioning different types of files by leveraging advanced NLP and image captioning models.


In [8]:
import io
from googleapiclient.http import MediaIoBaseDownload
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from PIL import Image
import PyPDF2
import docx
from mammoth import convert_to_markdown


class Summarizer():
    def __init__(self, service):
        self.service = service

    def download_file(self, file_id, file_name, file_type):
        try:
            request = self.service.files().get_media(fileId=file_id)
            fh = io.FileIO(file_name, 'wb')
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Download {int(status.progress() * 100)}% complete.")

            if file_type in ["jpg", "jpeg", "png"]:
                temp= self.image_captioner(file_name)
                print(f'The summary for {file_name} is {temp}')
                os.remove(file_name)
                return temp
            elif file_type in ["docx","msword"]:
                temp = self.doc_summarizer(file_name)
                print(f'The summary for {file_name} is {temp}')
                os.remove(file_name)
                return temp
            elif file_type == "pdf":
                temp= self.pdf_summarizer(file_name)
                print(f'The summary for {file_name} is {temp}')
                os.remove(file_name)
                return temp
            else:
                print('Couldnt create a summary')
                return 'Miscellaneous'
        except Exception as e:
            print(f"Error downloading or processing file: {str(e)}")
            return 'Error in processing'

    def pdf_summarizer(self, file):


        # Extract text from PDF
        def extract_text_from_pdf(pdf_path):
            text = ""
            try:
                with open(pdf_path, 'rb') as pdf_file:
                    reader = PyPDF2.PdfReader(pdf_file)
                    for page in reader.pages:
                        text += page.extract_text()
            except Exception as e:
                print(f"Error extracting text from PDF: {str(e)}")
            return text

        text = extract_text_from_pdf(file)
        if not text.strip():
            return "No text was extracted from the PDF."

        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        max_input_length = 1024
        if len(text) > max_input_length:
            text = text[:max_input_length]  # Truncate the text if it's too long
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        return summary

    def doc_summarizer(self, file):


        def extract_text_from_docx(docx_path):
            try:
                doc = docx.Document(docx_path)
                return "\n".join([para.text for para in doc.paragraphs])
            except Exception as e:
                print(f"Error extracting text from DOCX: {str(e)}")
                return ""

        file_ext = os.path.splitext(file)[1].lower()
        if file_ext == ".docx":
            text = extract_text_from_docx(file)
        else:
            return "Unsupported file format."

        if not text.strip():
            return "No text was extracted from the file."

        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        max_input_length = 1024
        if len(text) > max_input_length:
            text = text[:max_input_length]  # Truncate if too long
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        return summary

    def image_captioner(self, file):

        try:
            processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
            image = Image.open(file)
            inputs = processor(images=image, return_tensors="pt")
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)
            return caption
        except Exception as e:
            print(f"Error captioning image: {str(e)}")
            return "Error generating caption"

# Google Drive File Organizer with LLM-Based Categorization

This Python code integrates with Google Drive to list files in a folder, generate summaries using a file summarizer, classify the files using a large language model (LLM), and move them into categorized subfolders based on the classification.

### Key Components:

1. **Google Drive API Setup**:
   - The code handles authentication with Google Drive API using OAuth credentials (`token.json` and `credentials.json`).
   - The main folder and the destination folder are defined by their respective folder IDs.

2. **Listing Files from a Folder**:
   - The `list_files_in_folder` function recursively lists all files within a folder (and its subfolders) on Google Drive. It collects file metadata such as name, path, creation/modification dates, and MIME type.

3. **Summarization and Categorization**:
   - The `get_category` function uses the `Summarizer` class to generate summaries for supported file types (PDF, DOCX, images).
   - For each file, an LLM-based classifier (via the Groq API) categorizes the file based on its summary. The file is classified into one of the predefined categories like Accounting, Marketing, Operations, etc.

4. **File Movement**:
   - Once categorized, the file is moved to a folder named after the determined category. The `create_folder_if_not_exists` function ensures that the folder for each category is created if it doesn't already exist.
   - The `move_file` function moves the file to the appropriate subfolder within the destination folder.

5. **Process**:
   - For each file in the folder, the code fetches its summary, determines its category using the LLM, and moves the file into a corresponding category folder within the destination folder.


In [9]:
import json
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import json
#from summarizing import Summarizer
from groq import Groq

SCOPES = ['https://www.googleapis.com/auth/drive']
# Set your folder ID here

#folder_id = '1NNo5q_AcQuQqv_rqWhcD0XhXoTpZRc4N'
# Folder where data is present
folder_id = '1DosoFD-WtRC2CWaq-HhoMYxZfl1fhoXY'

# Folder where data is to be moved
folder_move_id = "1v-o0k9b5EjgKh6DCsu5ZYdNDUdhnvFsv"
creds = None

# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())

service = build("drive", "v3", credentials=creds)

def list_files_in_folder(service, folder_id, master_list, folder_path=""):
    # Query to get files and folders in a specific folder
    query = f"'{folder_id}' in parents and trashed = false"
    results = service.files().list(
        q=query,
        fields="nextPageToken, files(id, name, mimeType, createdTime, modifiedTime, parents)",
        supportsAllDrives=True,
        includeItemsFromAllDrives=True
    ).execute()

    items = results.get('files', [])

    for item in items:
        file_id = item['id']
        file_name = item['name']
        created_time = item.get('createdTime')
        modified_time = item.get('modifiedTime')
        mime_type = item.get('mimeType')

        # Check if the item is a folder or file
        if mime_type == 'application/vnd.google-apps.folder':
            # Recursively go through subfolders
            new_folder_path = f"{folder_path}/{file_name}"
            list_files_in_folder(service, file_id, master_list, new_folder_path)
        else:
            # File details
            file_path = f"{folder_path}/{file_name}"
            file_type = mime_type.split('/')[-1]  # Get the file type from mimeType
            master_list.append({
                'id': file_id,
                'name': file_name,
                'path': file_path,
                'mimeType': file_type,
                'createdTime': created_time,
                'modifiedTime': modified_time
            })
            print(f"File: {file_name}, Type: {file_type}, Path: {file_path}, Created: {created_time}, Modified: {modified_time}")
    return master_list


# Initialize the master list to store all files
master_list = []
# Start listing files from the root folder
list_of_files= list_files_in_folder(service, folder_id, master_list)


# LLM based classifier
def get_category_with_summary(summary):


    client = Groq(
        api_key="Enter Your API Key",
    )
    completion = client.chat.completions.create(
        model="llama-3.2-90b-text-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a File Categorizer"
            },
            {
                "role": "user",
                "content": "Given a text summary of a file that could be an image, pdf or a word document, categorize it into one of the following buckets-\n[Accounting,\nCuration,\nDevelopment (contributed revenue generation),\nEmployee resources (HR),\nBoard of Directors,\nMarketing,\nOperations, \nProgramming,\nResearch]\n\nThe output should only be the name of the most relevant bucket\n\n" + str(summary)
            }
        ],
        temperature=0.1,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )


    return completion.choices[0].message.content


def get_category(service, file_id, file_name, file_type):
    # Define keywords for each category
    summarizer = Summarizer(service)

    if file_type in ["pdf", "docx", "jpg", "jpeg", "png","msword"]:
        summary_text= summarizer.download_file(file_id, file_name, file_type)
        return get_category_with_summary(summary_text)
    else:
        categories = json.loads(open("tagging_map.json").read())
        for category, keywords in categories.items():
            if any(keyword in file_name for keyword in keywords):
                return category
            else:
                return "Misc"

## Updating a file
def create_folder_if_not_exists(service, name, parent_id=None):
    # Search for an existing folder with the same name
    query = f"mimeType='application/vnd.google-apps.folder' and name='{name}' and '{parent_id}' in parents and trashed = false"
    results = service.files().list(
        q=query,
        fields="files(id, name)",
        supportsAllDrives=True,
        includeItemsFromAllDrives=True
    ).execute()

    items = results.get('files', [])

    if items:
        # Folder already exists, return its ID
        return items[0]['id']

    # Folder doesn't exist, create a new one
    file_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    if parent_id:
        file_metadata['parents'] = [parent_id]

    folder = service.files().create(body=file_metadata, fields='id').execute()
    return folder.get('id')


def move_file(service, file_id, old_folder_id, new_folder_id):
    # Retrieve the existing parents to remove the file from old folders
    file = service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))

    # Move the file to the new folder
    service.files().update(
        fileId=file_id,
        addParents=new_folder_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()


for i in list_of_files:
    category = get_category(service, i['id'],i['name'], i['mimeType'])
    new_folder_id = create_folder_if_not_exists(service, category, parent_id=folder_move_id)
    move_file(service, i['id'], folder_id, new_folder_id)


File: save the date.pub, Type: octet-stream, Path: /Breakfast with Santa-20241013T043334Z-001/Breakfast with Santa/2009/save the date.pub, Created: 2024-10-13T17:15:56.304Z, Modified: 2024-10-13T04:34:16.000Z
File: b-s baskets 2004.pub, Type: octet-stream, Path: /Breakfast with Santa-20241013T043334Z-001/Breakfast with Santa/BwS 2004/b-s baskets 2004.pub, Created: 2024-10-13T17:15:57.449Z, Modified: 2024-10-13T04:34:16.000Z
File: Heritage Square Foundation table coverings for dry cleaning.doc, Type: msword, Path: /Breakfast with Santa-20241013T043334Z-001/Breakfast with Santa/BwS 2004/Heritage Square Foundation table coverings for dry cleaning.doc, Created: 2024-10-13T17:15:59.108Z, Modified: 2024-10-13T04:34:16.000Z
File: basket database 2004.doc, Type: msword, Path: /Breakfast with Santa-20241013T043334Z-001/Breakfast with Santa/BwS 2004/basket database 2004.doc, Created: 2024-10-13T17:15:57.449Z, Modified: 2024-10-13T04:34:16.000Z
File: d & d ty.doc, Type: msword, Path: /Breakfast w

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

The summary for 5. Sanborn Map Wall Text 2.docx is The first land survey for a permanent settlement was in 1867. Fueled by Jack Swilling’s irrigation ditch, the tiny town of Phoenix, Arizona, had humble, capitalistic beginnings.
Download 100% complete.
The summary for hoop and stick.jpg is a boy in a costume and hat with a sword
Download 100% complete.
The summary for Main Wall Sign Design.docx is No text was extracted from the file.
Download 100% complete.
The summary for 3. GPS and Square Map.docx is No text was extracted from the file.
Download 100% complete.
The summary for S-H Bungalow for YAH Intro.JPG is a map of the area
Download 100% complete.
The summary for 13. Diversity Wall Chinese.docx is Chinese immigrants first came to Phoenix to work in the mines and railroads. Many stayed to develop commercial enterprises. Early community leaders encouraged their countrymen to disperse settlement to avoid racial conflict.
Download 100% complete.
The summary for 12. Diversity Wall Afri

Your max_length is set to 130, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


The summary for 7. Sanborn Map Wall Text 4.docx is By 1878 Phoenix boosters were delighted that their enterprise was taking off. The once humble settlement appeared more ‘American’ as the new fired brick factory provided an alternative to the coarse adobe huts.
Download 100% complete.
The summary for Plexi Wall Sign.docx is No text was extracted from the file.
Download 100% complete.
The summary for Mapping Smalls.docx is No text was extracted from the file.
Download 100% complete.
The summary for Rosson Block 14 (1885) and Section 35 Deeds.docx is This Indenture, made the 12th day of March, in the year of our Lord, one thousand eight hundred and eighty five, between James Addison Reavis of the Territory of Arizona, party of the first part, and Mrs. Flora B. Rosson of the City of Phoenix, County of Maricopa, of said Territory. Witnesseth, that the said party of. the first. part, for and in consideration of the sum of one hundred and fifty ($150) 00/100 Dollars, lawful. money of the Uni

Your max_length is set to 130, but your input_length is only 19. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)


The summary for _You Are Here_Tech Options.docx is "You Are Here: Phoenix Then and Now" will be on display at the Phoenix Museum of Art and Science. The exhibition will run through the end of the year. The museum will be open to the general public.
Download 100% complete.
The summary for Rosson Mine Deed.docx is This Indenture, made the Eighth “8th day of November in the year of our Lord, One Thousand Eight Hundred and Eighty two, between Dr R. L. Rosson of the first part of Phoenix Maricopa County Arizona and the party of the second part. Witnesseth: That the said party of. first part, for and in consideration of the sum of One [Doll-lar] Dollar lawful money of the United States of America, to me in hand.
